In [ ]:

# --- Imports ---
import pandas as pd
import numpy as np

print("--- Starting Part 1: Data Merging and Cleaning ---")

# --- Configuration: File Names and Loading Parameters ---
file_ieg = 'ieg_wbg.csv'
file_ops = 'projects_operations.csv'

--- Starting Part 1: Data Merging and Cleaning ---


In [ ]:
# --- V V V --- VERIFY LOADING PARAMETERS --- V V V ---
# Setting header=0 based on user providing actual headers. Verify delimiter.
delimiter1 = ',' # For ieg_wbg.csv
header_row1 = 0
delimiter2 = ',' # For projects_operations.csv
header_row2 = 0


In [ ]:
# --- Standardization Function ---
def standardize_cols(df):
    """Converts column names to lowercase, replaces spaces/special chars with underscores."""
    df.columns = df.columns.astype(str).str.strip().str.lower()
    df.columns = df.columns.str.replace(' ', '_', regex=False)
    df.columns = df.columns.str.replace(r'[ /$.()]+', '_', regex=True) # More comprehensive symbol replacement
    df.columns = df.columns.str.replace(r'[^a-z0-9_]+', '', regex=True)
    df.columns = df.columns.str.strip('_').str.replace(r'_+', '_', regex=True)
    return df

In [ ]:
# --- Step 1: Load Datasets ---
print(f"\nLoading Dataset 1: {file_ieg}")
try:
    df_ieg = pd.read_csv(file_ieg, header=header_row1, delimiter=delimiter1, low_memory=False) # Added low_memory=False for potentially mixed types
    print(f"Loaded {file_ieg} successfully. Shape: {df_ieg.shape}")
except Exception as e: print(f"Error loading {file_ieg}: {e}"); exit()

print(f"\nLoading Dataset 2: {file_ops}")
try:
    df_ops = pd.read_csv(file_ops, header=header_row2, delimiter=delimiter2, low_memory=False)
    print(f"Loaded {file_ops} successfully. Shape: {df_ops.shape}")
except Exception as e: print(f"Error loading {file_ops}: {e}"); exit()



Loading Dataset 1: ieg_wbg.csv
Loaded ieg_wbg.csv successfully. Shape: (6314, 21)

Loading Dataset 2: projects_operations.csv
Loaded projects_operations.csv successfully. Shape: (18841, 57)


In [ ]:
# --- Step 2: Standardize Columns ---
df_ieg = standardize_cols(df_ieg.copy())
df_ops = standardize_cols(df_ops.copy())
print("\nStandardized columns for both datasets.")
print(f"\n--- >>> CHECK THESE STANDARDIZED COLUMNS CAREFULLY <<< ---")
print(f"Columns in {file_ieg} (standardized):", df_ieg.columns.tolist())
print(f"\nColumns in {file_ops} (standardized):", df_ops.columns.tolist())
print("--- >>>> Verify the variable assignments below match these lists <<< ---")



Standardized columns for both datasets.

--- >>> CHECK THESE STANDARDIZED COLUMNS CAREFULLY <<< ---
Columns in ieg_wbg.csv (standardized): ['as_of_date', 'project_id', 'project_name', 'wb_region', 'country_economy', 'country_economy_lending_group', 'country_economy_fcs_status', 'country_economy_fcs_lending_group', 'practice_group', 'global_practice', 'agreement_type', 'lending_instrument_type', 'approval_fy', 'final_closing_fy', 'evaluation_type', 'outcome', 'quality_at_entry', 'quality_of_supervision', 'bank_performance', 'me_quality', 'evaluation_fy']

Columns in projects_operations.csv (standardized): ['id', 'regionname', 'countryname', 'prodline', 'lendinginstr', 'lendinginstrtype', 'envassesmentcategorycode', 'supplementprojectflg', 'productlinetype', 'projectstatusdisplay', 'status', 'project_name', 'boardapprovaldate', 'board_approval_month', 'closingdate', 'lendprojectcost', 'ibrdcommamt', 'idacommamt', 'totalamt', 'grantamt', 'borrower', 'impagency', 'url', 'projectdoc', 'maj

In [ ]:
# --- Step 3: Define Key Columns & Merge Key ---
# >>> IMPORTANT: Verify these standardized names match the printed lists ABOVE <<<

# Project ID Columns (Based on user-provided headers 'Project ID' and 'id')
id_col_ieg = 'project_id'
id_col_ops = 'id'

# Define other columns needed for ANY hypothesis, using expected standardized names
# From IEG file (based on user headers):
outcome_col = 'outcome' # Raw header was 'Outcome' - ASSUME THIS HAS TEXT RATINGS NOW
qat_entry_col = 'quality_at_entry'
lending_group_col = 'country_economy_lending_group'
fcs_status_col = 'country_economy_fcs_status' # New context variable for potential H6 variant
practice_group_col = 'practice_group' # H3 Candidate
global_practice_col = 'global_practice' # H3 Candidate
approval_fy_col = 'approval_fy' # Year column from IEG
ieg_instr_type_col = 'lending_instrument_type' # H3 Candidate (potentially duplicate name)


In [ ]:
# From Ops file (based on user headers):
cost_col = 'lendprojectcost'
country_col = 'countryname'
region_col = 'regionname'
ops_instr_type_col = 'lendinginstrtype' # H3 Candidate (potentially duplicate name)
ops_approval_date_col = 'boardapprovaldate' # Alternative date column

print("\nKey columns identified (verify names):")
print(f"  IEG ID: {id_col_ieg}, Ops ID: {id_col_ops}")
print(f"  Outcome: {outcome_col}, QAT Entry: {qat_entry_col}")
print(f"  Cost: {cost_col}, Country: {country_col}, Region: {region_col}")
print(f"  Lending Group: {lending_group_col}, FCS Status: {fcs_status_col}")
print(f"  Year Source: {approval_fy_col} (from IEG)")
print(f"  Project Types (IEG): {practice_group_col}, {global_practice_col}, {ieg_instr_type_col}")
print(f"  Project Types (Ops): {ops_instr_type_col}")



Key columns identified (verify names):
  IEG ID: project_id, Ops ID: id
  Outcome: outcome, QAT Entry: quality_at_entry
  Cost: lendprojectcost, Country: countryname, Region: regionname
  Lending Group: country_economy_lending_group, FCS Status: country_economy_fcs_status
  Year Source: approval_fy (from IEG)
  Project Types (IEG): practice_group, global_practice, lending_instrument_type
  Project Types (Ops): lendinginstrtype


In [ ]:
# --- Step 4: Prepare for Merge ---
merge_key = 'project_id_merged'
# Verify and Rename ID columns
if id_col_ieg not in df_ieg.columns: print(f"Error: Col '{id_col_ieg}' not in {file_ieg}."); exit()
df_ieg.rename(columns={id_col_ieg: merge_key}, inplace=True)
if id_col_ops not in df_ops.columns: print(f"Error: Col '{id_col_ops}' not in {file_ops}."); exit()
df_ops.rename(columns={id_col_ops: merge_key}, inplace=True)
print(f"Renamed ID columns to '{merge_key}'.")


Renamed ID columns to 'project_id_merged'.


In [ ]:
# --- Step 5: Select ALL potentially useful columns and Merge ---
# Select all defined columns plus the key from each dataframe where they exist
cols_to_keep_ieg = list(set([merge_key, outcome_col, qat_entry_col, lending_group_col, fcs_status_col,
                             practice_group_col, global_practice_col, approval_fy_col, ieg_instr_type_col]))
cols_to_keep_ops = list(set([merge_key, cost_col, country_col, region_col, ops_instr_type_col, ops_approval_date_col]))

# Ensure selected columns actually exist in the dataframes before trying to select
cols_to_select_ieg = [col for col in cols_to_keep_ieg if col in df_ieg.columns]
cols_to_select_ops = [col for col in cols_to_keep_ops if col in df_ops.columns]
# Avoid selecting the same column name twice (besides merge key) - prioritize IEG version if name collides
cols_to_select_ops = [col for col in cols_to_select_ops if col not in cols_to_select_ieg or col == merge_key]

print(f"\nSelecting columns from {file_ieg}: {cols_to_select_ieg}")
print(f"Selecting columns from {file_ops}: {cols_to_select_ops}")

# Perform INNER merge (keeps only matching projects, ~6k rows)
print(f"\nPerforming inner merge on '{merge_key}'...")
df_merged_clean = pd.merge(
    df_ieg[cols_to_select_ieg],
    df_ops[cols_to_select_ops],
    on=merge_key,
    how='inner',
    suffixes=('_ieg', '_ops') # Add suffix if same column name selected from both by mistake
)
print(f"\nMerge complete. Shape of merged data: {df_merged_clean.shape}")
print(f"Columns in merged data: {df_merged_clean.columns.tolist()}")
if df_merged_clean.empty: print("\nError: Merged DataFrame is empty."); exit()




Selecting columns from ieg_wbg.csv: ['outcome', 'country_economy_lending_group', 'global_practice', 'lending_instrument_type', 'practice_group', 'project_id_merged', 'quality_at_entry', 'approval_fy', 'country_economy_fcs_status']
Selecting columns from projects_operations.csv: ['lendinginstrtype', 'boardapprovaldate', 'countryname', 'lendprojectcost', 'regionname', 'project_id_merged']

Performing inner merge on 'project_id_merged'...

Merge complete. Shape of merged data: (6163, 14)
Columns in merged data: ['outcome', 'country_economy_lending_group', 'global_practice', 'lending_instrument_type', 'practice_group', 'project_id_merged', 'quality_at_entry', 'approval_fy', 'country_economy_fcs_status', 'lendinginstrtype', 'boardapprovaldate', 'countryname', 'lendprojectcost', 'regionname']


In [ ]:
# --- Step 6: Initial Cleaning of Merged Data ---
print("\nPerforming initial cleaning...")

# Handle potential duplicate columns from merge (example for lending instrument type)
# If both _ieg and _ops versions exist, decide which to keep (e.g., prioritize _ieg)
ieg_type_final = ieg_instr_type_col + '_ieg' # Check if suffix was added
ops_type_final = ops_instr_type_col + '_ops' # Check if suffix was added
final_type_col = 'lending_instrument_type' # Desired final name

if ieg_type_final in df_merged_clean.columns and ops_type_final in df_merged_clean.columns:
    print(f"Handling duplicate type columns: Keeping '{ieg_type_final}', dropping '{ops_type_final}'")
    df_merged_clean[final_type_col] = df_merged_clean[ieg_type_final]
    df_merged_clean.drop(columns=[ieg_type_final, ops_type_final], inplace=True)
elif ieg_type_final in df_merged_clean.columns:
     df_merged_clean.rename(columns={ieg_type_final: final_type_col}, inplace=True)
elif ops_type_final in df_merged_clean.columns:
      df_merged_clean.rename(columns={ops_type_final: final_type_col}, inplace=True)
elif ieg_instr_type_col in df_merged_clean.columns: # If no suffix was added but exists
     df_merged_clean.rename(columns={ieg_instr_type_col: final_type_col}, inplace=True)
elif ops_instr_type_col in df_merged_clean.columns: # If no suffix was added but exists
     df_merged_clean.rename(columns={ops_instr_type_col: final_type_col}, inplace=True)
# Add similar logic if other columns might be duplicated (e.g., region, country if names were same)

# Convert Cost to numeric (handle errors)
if cost_col in df_merged_clean.columns:
    df_merged_clean[cost_col] = df_merged_clean[cost_col].astype(str).str.replace(r'[$,]', '', regex=True)
    df_merged_clean[cost_col] = pd.to_numeric(df_merged_clean[cost_col], errors='coerce')
    print(f"Converted '{cost_col}' to numeric.")

# Convert Year to numeric (handle errors)
if approval_fy_col in df_merged_clean.columns:
    df_merged_clean[approval_fy_col] = pd.to_numeric(df_merged_clean[approval_fy_col], errors='coerce')
    # Keep as float for now to preserve NaNs, convert to Int AFTER dropping NAs if needed
    # df_merged_clean[approval_fy_col] = df_merged_clean[approval_fy_col].astype('Int64') # Use nullable integer
    print(f"Converted '{approval_fy_col}' to numeric.")

# Ensure key categorical columns are strings and stripped
for col in [outcome_col, qat_entry_col, lending_group_col, fcs_status_col,
            practice_group_col, global_practice_col, final_type_col,
            country_col, region_col]:
    if col in df_merged_clean.columns:
        df_merged_clean[col] = df_merged_clean[col].astype(str).str.strip()

print("Performed initial type conversions.")



Performing initial cleaning...
Converted 'lendprojectcost' to numeric.
Converted 'approval_fy' to numeric.
Performed initial type conversions.


In [ ]:
# Report Null counts AFTER merge and initial cleaning
print("\nNull value counts in key columns of merged data:")
key_cols_final = [col for col in [merge_key, outcome_col, cost_col, country_col, region_col, approval_fy_col, qat_entry_col, lending_group_col, fcs_status_col, final_type_col, practice_group_col, global_practice_col] if col in df_merged_clean.columns]
print(df_merged_clean[key_cols_final].isnull().sum())



Null value counts in key columns of merged data:
project_id_merged                 0
outcome                           0
lendprojectcost                  49
countryname                       0
regionname                        0
approval_fy                       0
quality_at_entry                  0
country_economy_lending_group     0
country_economy_fcs_status        0
lending_instrument_type           0
practice_group                    0
global_practice                   0
dtype: int64


In [ ]:
# --- Step 6b: Reorder Columns (Move Project ID to Front) --- # ADD THIS BLOCK
print(f"\nReordering columns to place '{merge_key}' first...")
# Get the list of current column names
current_cols = df_merged_clean.columns.tolist()
# Make sure the merge key column exists
if merge_key in current_cols:
    # Create the new desired column order: merge key first, then all others
    new_col_order = [merge_key] + [col for col in current_cols if col != merge_key]
    # Apply the new order to the DataFrame
    df_merged_clean = df_merged_clean[new_col_order]
    print("Columns reordered successfully.")
    # print(f"First few columns now: {df_merged_clean.columns.tolist()[:5]}") # Optional: view first few cols
else:
    print(f"Warning: Merge key '{merge_key}' not found in columns. Cannot reorder.")



Reordering columns to place 'project_id_merged' first...
Columns reordered successfully.


In [ ]:
export_filename = 'wbg_merged_cleaned_for_analysis.csv'
try:
    # Export the DataFrame with the new column order
    df_merged_clean.to_csv(export_filename, index=False) # index=False prevents writing row numbers as a column
    print(f"\nSuccessfully exported cleaned and merged data (with '{merge_key}' as first column) to: {export_filename}")
    print(f"Final shape of exported data: {df_merged_clean.shape}")
except Exception as e:
    print(f"\nError exporting data to CSV: {e}")

print("\n--- Part 1 Complete ---")


Successfully exported cleaned and merged data (with 'project_id_merged' as first column) to: wbg_merged_cleaned_for_analysis.csv
Final shape of exported data: (6163, 14)

--- Part 1 Complete ---


In [ ]:
from google.colab import drive
drive.mount('/content/drive')